In [1]:
# IMPORTS
import os
import pandas as pd
import numpy as np
pd.set_option('display.float_format', '{:.10f}'.format)

In [3]:
# Load files
ids = pd.read_excel('C:/Users/alexb/OneDrive/DP3/DP3_IDS.xlsx')
log = pd.read_excel('C:/Users/alexb/OneDrive/DP3/VAS_Log.xlsx')
log.rename(columns = {'Monitor ID':'ID', 'Location/participant':'PPN', 'Downoad data':'download', 'Deployment date': 'deploy', 'Return date':'return'}, inplace = True)


In [4]:
#Clean & convert to lower
log_lower = log.map(lambda x: x.lower() if isinstance(x, str) else x)
log = log_lower
log_yes = log[log['download'].str.contains('yes', case=False, na=False)]
log = log_yes.copy() # Only yes downloads
print(sum(pd.to_datetime(log['deploy']).isna()))
log['deploy'] = pd.to_datetime(log['deploy'])
log['deploy'] = log['deploy'].astype('int64') // 10**9 #Convert time to unix epoch
print(sum(pd.to_datetime(log['return']).isna()))
log['return'] = pd.to_datetime(log['return'])
log['return'] = log['return'].astype('int64') // 10**9 #Convert time to unix epoch


# Pass to dict structure for key:value mapping
log_dict_ids = log.groupby('ID')['PPN'].apply(list).to_dict() # Monitor : PPN
log_dict_times = log.set_index('PPN')[['deploy', 'return']].apply(tuple, axis = 1).to_dict() # PPN : (deploy, return)
log_dict_times

0
0


{'dp3-0001': (1614211200, 1623283200),
 'dp3-0003': (1616976000, 1623974400),
 'dp3-0005': (1625616000, 1629763200),
 'dp3-0006': (1616025600, 1620864000),
 'dp3-0008': (1616457600, 1621296000),
 'dp3-0009': (1617235200, 1621987200),
 'dp3-0010': (1618185600, 1623369600),
 'dp3-0011': (1619136000, 1626134400),
 'dp3-0012': (1617840000, 1622764800),
 'dp3-0013': (1619136000, 1623974400),
 'dp3-0015': (1618790400, 1622592000),
 'dp3-0018': (1619654400, 1624838400),
 'dp3-0019': (1620259200, 1624320000),
 'dp3-0020': (1620864000, 1625788800),
 'dp3-0022': (1623628800, 1627344000),
 'dp3-0023': (1623196800, 1635984000),
 'dp3-0024': (1624838400, 1628812800),
 'dp3-0027': (1628467200, 1632182400),
 'dp3-0028': (1629072000, 1633392000),
 'dp3-0036': (1629331200, 1633651200),
 'dp3-0050': (1632787200, 1637539200),
 'dp3-0062': (1645747200, 1649289600),
 'dp3-0070': (1644969600, 1650326400),
 'dp3-0097': (1651017600, 1655251200),
 'dp3-0101': (1642118400, 1652400000),
 'dp3-0105': (1652659200,

In [69]:
# Data Scrubbing
# Loop through each file in the directory, get the keys associated with that 

path = 'C:/Users/alexb/OneDrive/DP3/magee-pm-data-2024-09/magee-pm-data-2024-09'

for filename in os.listdir(path):
    # Check if the path is a file
    file_path = os.path.join(path, filename)
    if os.path.isfile(file_path):
        #Check ID through filename
        print(f'Processing file: {repr(filename[0:6])}')


        # Get monitor data from current file
        monitor_data = pd.read_csv(file_path)
        # set time to index for index based slicing
        monitor_data = monitor_data.set_index('unix_epoch')
        # Select only neeed readout variables
        monitor_data = monitor_data[['temperature', 'humidity', 'pressure', 'voc']]
        

        #Get associated ppns 
        airviz_id = filename[0:6]
        ppn_list = log_dict_ids.get(airviz_id) 
        print(ppn_list)
        #Init ppn frame
        ppn_save_data = None
        #Get UTC times for each ppn
        if ppn_list != None:
            for ppn in ppn_list:
                start_time, end_time = log_dict_times[ppn]
                print(start_time, end_time)

                ppn_save_data = monitor_data.loc[float(start_time):float(end_time)]
                save_name = '-'.join([ppn , airviz_id, '.csv'])
                ppn_save_data.to_csv(''.join(['C:/Users/alexb/OneDrive/DP3/PPN - Monitor_ID - Data/', save_name]), index_label='unix_epoch' )
        


Processing file: '06e5ac'
['dp3-0152']
1652227200 1657065600
Processing file: '1748fa'
['dp3-0008', 'dp3-0023', 'dp3-0036', 'dp3-0142', 'dp3-0255', 'dp3-0308', 'dp3-0353', 'dp3-0405', 'dp3-0425']
1616457600 1621296000
1623196800 1635984000
1629331200 1633651200
1649635200 1654041600
1669766400 1675123200
1679529600 1684108800
1684886400 1690416000
1698624000 1702857600
1703548800 1708473600
Processing file: '2441da'
['dp3-0006', 'dp3-0123', 'dp3-0226', 'dp3-0295', 'dp3-0330']
1616025600 1620864000
1649203200 1654128000
1665619200 1670544000
1687305600 1692144000
1681776000 1686700800
Processing file: '268af8'
['dp3-0113', 'dp3-0160', 'dp3-0253', 'dp3-0314', 'dp3-0357', 'dp3-0388', 'dp3-0421']
1654732800 1660867200
1663113600 1668729600
1669680000 1674518400
1680652800 1685404800
1686009600 1691366400
1692316800 1697068800
1733616000 1706400000
Processing file: '291d69'
['dp3-0156', 'dp3-0169']
1651536000 1656374400
1654560000 1660003200
Processing file: '2c564e'
['dp3-0001', 'dp3-0024'

In [31]:
# Load in ppns where multiple monitors were given
path = 'C:/Users/alexb/OneDrive/DP3/PPN - Monitor_ID - Data'

#Init dictionary
file_groups = {}

segments = {}
#Since files are loaded by ppn sorting, we do not need to worry about holding more than one dataset at a time
# PPNs have a max of 2 monitors in sequence so a simple hold and check will work
for filename in os.listdir(path):
    if filename.endswith(".csv"):
        # Extract file name ID 
        file_parts = filename.split('-')
        


        file_id = '-'.join((file_parts[0], file_parts[1]))
        
        monitor_id = file_parts[2]

        key = file_id
        
        

        # Load the CSV into a DataFrame
        df = pd.read_csv(os.path.join(path, filename))
        
        # Check if the file ID exists in the dictionary, if not, initialize a list
        if key in file_groups:
            # Concatenate if the ID exists
            file_groups[key].append(df)
            segments[key].append(monitor_id)
        else:
            # Create a new list for this file ID
            file_groups[key] = [df]     
            segments[key] = [monitor_id]

In [37]:
# Now, concatenate all DataFrames that share the same file ID
for key, df_list in file_groups.items():
    # Concatenate the DataFrames for each file_id
    concatenated_df = pd.concat(df_list, ignore_index=True)
    concatenated_df.sort_values(by = 'unix_epoch', ascending=True, inplace = True)
    filename_out = f'{key}_{segments[key][0:]}.csv'
    print(filename_out)

    # Save the concatenated DataFrame (optional)
    concatenated_df.to_csv(os.path.join(path, filename_out), index=False)

    print(f"Concatenated files for ID: {key}") 

dp3-0001_['2c564e'].csv
Concatenated files for ID: dp3-0434
dp3-0003_['4b81c6'].csv
Concatenated files for ID: dp3-0434
dp3-0005_['c6b87f'].csv
Concatenated files for ID: dp3-0434
dp3-0006_['2441da'].csv
Concatenated files for ID: dp3-0434
dp3-0008_['1748fa'].csv
Concatenated files for ID: dp3-0434
dp3-0009_['4b81c6', 'c9fb6e'].csv
Concatenated files for ID: dp3-0434
dp3-0010_['b67138'].csv
Concatenated files for ID: dp3-0434
dp3-0011_['a1fb71'].csv
Concatenated files for ID: dp3-0434
dp3-0012_['4b33be', 'd14457'].csv


C:\Users\alexb\AppData\Local\Temp\ipykernel_4936\2525020558.py:4: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  concatenated_df = pd.concat(df_list, ignore_index=True)


Concatenated files for ID: dp3-0434
dp3-0013_['c30454'].csv
Concatenated files for ID: dp3-0434
dp3-0018_['8e5d53'].csv
Concatenated files for ID: dp3-0434
dp3-0019_['c6b87f'].csv
Concatenated files for ID: dp3-0434
dp3-0020_['4b33be'].csv
Concatenated files for ID: dp3-0434
dp3-0022_['d14457'].csv
Concatenated files for ID: dp3-0434
dp3-0023_['1748fa'].csv
Concatenated files for ID: dp3-0434
dp3-0024_['2c564e'].csv
Concatenated files for ID: dp3-0434
dp3-0027_['d14457'].csv
Concatenated files for ID: dp3-0434
dp3-0028_['bbae56'].csv
Concatenated files for ID: dp3-0434
dp3-0036_['1748fa'].csv
Concatenated files for ID: dp3-0434
dp3-0050_['8e5d53'].csv
Concatenated files for ID: dp3-0434
dp3-0062_['2c564e'].csv
Concatenated files for ID: dp3-0434
dp3-0097_['8eceb4'].csv
Concatenated files for ID: dp3-0434
dp3-0101_['b18aab'].csv
Concatenated files for ID: dp3-0434
dp3-0105_['8e5d53'].csv
Concatenated files for ID: dp3-0434
dp3-0108_['47e871'].csv
Concatenated files for ID: dp3-0434
dp3-